In [5]:
from PIL import Image
import requests
import numpy as np
from PIL import Image

import pandas as pd 
import geopandas as gpd
# import osmnx as ox
# import networkx as nx

import pyproj
from shapely.ops import transform

import shapely
from shapely import Point

# import boto3
import os
import django

pd.set_option('display.precision', 2)

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "water.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from water.models import ReservoirState, Reservoir, ReservoirStateSerializer, RainFall

from water.utils import parse as p

In [23]:
filename_all = '../../../../data/datasets/all_parsed_cleaned.csv'
df_all_raw = pd.read_csv(filename_all).sort_values(['province', 'reservoir', 'ds'])

In [24]:
capacities = df_all_raw.groupby(['province', 'reservoir'])['capacity_hm3'].agg(['first', 'nunique']).reset_index()
capacities = capacities[capacities['nunique'] ==1].copy()
assert capacities['nunique'].max() == 1
capacities = capacities.query('province=="malaga"')
capacities

df_selected = df_all_raw[df_all_raw.reservoir.isin(capacities.reservoir.unique())]

In [18]:
ReservoirState.objects.all().delete()
Reservoir.objects.all().delete()

(105, {'water.RainFall': 100, 'water.Reservoir': 5})

In [25]:
ReservoirState.objects.all().delete()
Reservoir.objects.all().delete()
num_states = 100

for _, row in capacities.iterrows():
    reservoir = Reservoir.objects.create(name=row['reservoir'], capacity=row['first'])
    reservoir.save()
    

print(len(df_selected))
    
for _, row in df_selected.head(num_states).iterrows():
    reservoir = Reservoir.objects.get(name=row['reservoir'])
    state = ReservoirState.objects.create(
        reservoir=reservoir,
        date=row['ds'],
        volume=row['stored_hm3'],
    )
    state.save()
    
ReservoirState.objects.all().count()

2859


100

In [40]:
df_all = p.add_cols(df_selected)
df_all['date_lag'] = df_all.groupby(['province', 'reservoir'])['date'].shift(1)
df_all['date_diff'] = (df_all.date - df_all.date_lag).dt.days

cols = ['rainfallsince', 'stored_hm3', 'capacity_hm3']
for var in ['rainfallsince', 'stored_hm3']:
    df_all[f'{var}_diff'] = df_all.groupby(['province', 'reservoir'])[var].diff()
    df_all[f'{var}_diff_0'] = df_all[f'{var}_diff']
    for lags in range(1, 10):
        df_all[f'{var}_diff_{lags}'] = df_all.groupby(['province', 'reservoir'])[f'{var}_diff'].shift(lags)
        
len(df_all.query('date_diff == 1'))
len(df_all)

2859

In [42]:
# Delete all rainfall objects
RainFall.objects.all().delete()

# Get all reservoirs with state data
reservoirs = Reservoir.objects.annotate(
        num_states=Count("reservoir_reservoirstate")
    ).filter(num_states__gt=0)

reservoir_names = reservoirs.values_list('name', flat=True)

nums = 100

for _, row in df_all[df_all.reservoir.isin(reservoir_names)].query('date_diff == 1').head(nums).iterrows():
    reservoir = Reservoir.objects.get(name=row['reservoir'])
    rainfall = RainFall.objects.create(
        date=row['ds'],
        reservoir=reservoir,
        amount=row['rainfallsince_diff']
    )
RainFall.objects.all().count()

100

In [43]:
reservoir_names

<QuerySet ['conde de guadalhorce']>